# F1 Win Prediction Project
#### Alex Boardman - BrainStation

### Areas to Fix

- **Data Types**: Ensure that each column has the appropriate data type for the kind of data it contains. For instance, categorical data should not be typed as numeric and vice versa. If any columns are meant to be categorical or date/time but are currently recognized as 'object' or 'int64', they should be converted to the proper data type.

- **Missing Data**: Your dataset contains columns with high percentages of missing values, such as 'team_development_rank_last_year' and 'status_finished_last_race'. You need to decide how to handle these, whether by imputation, deletion, or acquisition of more data if possible. For columns with a small amount of missing data, imputation might be feasible, while for those with a large percentage, it might be more appropriate to consider dropping the column.

- **Duplicate Rows**: Check for any duplicate rows that might skew your analysis. If duplicates are not meaningful for your study, they should be removed.

- **Uniqueness of Data**: Some columns like 'raceId' and 'driverId' are expected to have a high degree of uniqueness and serve as identifiers. Other columns that should normally have a diverse set of values but show a high degree of similarity (low uniqueness) may not be very informative and could potentially be candidates for removal.

- **Data Range**: Verify the range of values in numerical columns. For instance, if 'year' has a minimum value that's in the future or a past date that's not plausible, these could be data entry errors. Check for outliers that don't make sense within the context of the data.

- **Consistency**: Ensure that the data is consistent throughout the dataset. For example, if 'country' and 'nationality_of_circuit' are supposed to represent the same information, they should be consistent and possibly merged if they are redundant.

- **Correctness**: For columns with 'inf' values for uniqueness, ensure they are correctly calculated. An 'inf' value might indicate a division by zero error, suggesting that the column might be entirely unique or entirely composed of a single value, each of which has different implications.

- **Data Integrity**: Ensure that related columns correctly reflect relationships in the data. For example, 'number_of_pit_stops' should correlate with 'average_time_lost_in_pits' in a way that makes sense.

- **Normalization/Standardization**: For machine learning purposes, you may need to standardize or normalize numerical data to ensure that the scale of the data does not unduly influence the model.

## Data Preprocessing

### Importing Libraries and Notebook Setup

In [31]:
# Install libraries
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [32]:
process_df = pd.read_csv('C:/Users/Alex/OneDrive/BrainStation/Data_Science_Bootcamp/Capstone_Project/capstone-Aboard89/data/data_analysis.csv')

In [33]:
process_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11235 entries, 0 to 11234
Data columns (total 36 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Index                             11235 non-null  int64  
 1   resultId                          11235 non-null  int64  
 2   raceId                            11235 non-null  int64  
 3   year                              11235 non-null  int64  
 4   race                              11235 non-null  object 
 5   country                           11235 non-null  object 
 6   nationality_of_circuit            11235 non-null  object 
 7   driverId                          11235 non-null  int64  
 8   number                            11235 non-null  int64  
 9   driver_name                       11235 non-null  object 
 10  F2_champion                       11235 non-null  int64  
 11  Former_F1_World_Champion          11235 non-null  int64  
 12  Nati

In [34]:
pd.set_option('display.max_columns', None)
process_df.head()

,Index,resultId,raceId,year,race,country,nationality_of_circuit,driverId,number,driver_name,F2_champion,Former_F1_World_Champion,Nationality,home_race,constructorId,constructor,engine_manufacturer,constructor_nationality,number.1,starting_grid_position,positionOrder,points,points_in_previous_race,laps,laps_completed_in_previous_races,time,milliseconds,fastestLap_ms,fastest_lap_from_last_race,number_of_pit_stops,average_time_lost_in_pits,statusId,status,major_regulation_change,laps_in_previous_race,race_win
0,1,4721,240,1995,Brazilian Grand Prix,brazil,Brazilian,30,1,Michael Schumacher,0,1,German,0,22,Benetton,Renault,Italian,1,2,1,10.0,NaN,71,NaN,01:38:34.154000,5914154,81009,NaN,3,31.83,1,Finished,0,NaN,1
1,1,4724,240,1995,Brazilian Grand Prix,brazil,Brazilian,57,8,Mika Häkkinen,0,0,Finnish,0,1,McLaren,Mercedes,British,8,7,4,3.0,NaN,70,NaN,\N,\N,Not Found,NaN,0,0.00,11,+1 Lap,0,NaN,0
2,1,4746,240,1995,Brazilian Grand Prix,brazil,Brazilian,94,23,Pierluigi Martini,0,0,Italian,0,18,Minardi,Ford,Italian,23,17,26,0.0,NaN,0,NaN,\N,\N,Not Found,NaN,0,0.00,6,Gearbox,0,NaN,0
3,1,4745,240,1995,Brazilian Grand Prix,brazil,Brazilian,44,26,Olivier Panis,0,0,French,0,27,Ligier,Mugen-Honda,French,26,10,25,0.0,NaN,0,NaN,\N,\N,Not Found,NaN,0,0.00,4,Collision,0,NaN,0
4,1,4744,240,1995,Brazilian Grand Prix,brazil,Brazilian,49,30,Heinz-Harald Frentzen,0,0,German,0,15,Sauber,Ford,Swiss,30,14,24,0.0,NaN,10,NaN,\N,\N,84001,NaN,0,0.00,10,Electrical,0,NaN,0


Here you can add sections like:

- Renaming columns
- Drop Redundant Columns
- Changing Data Types
- Dropping Duplicates
- Handling Missing Values
- Handling Unreasonable Data Ranges
- Feature Engineering / Transformation

Use `assert` where possible to show that preprocessing is done.

### Rename Columns

In [35]:
# # Rename columns to snake_case
# df = clean_columns(df, replace={})

In [36]:
# # Rename columns
columns_to_rename = {'Index': 'race_index'}
process_df.rename(columns=columns_to_rename, inplace=True)

In [37]:
# # Verify columns are renamed
process_df.columns

Index(['race_index', 'resultId', 'raceId', 'year', 'race', 'country',
       'nationality_of_circuit', 'driverId', 'number', 'driver_name',
       'F2_champion', 'Former_F1_World_Champion', 'Nationality', 'home_race',
       'constructorId', 'constructor', 'engine_manufacturer',
       'constructor_nationality', 'number.1', 'starting_grid_position',
       'positionOrder', 'points', 'points_in_previous_race', 'laps',
       'laps_completed_in_previous_races', 'time', 'milliseconds',
       'fastestLap_ms', 'fastest_lap_from_last_race', 'number_of_pit_stops',
       'average_time_lost_in_pits', 'statusId', 'status',
       'major_regulation_change', 'laps_in_previous_race', 'race_win'],
      dtype='object')

### Drop Redundant Columns

In [38]:
# Define the columns to be dropped
cols_to_drop = [
    "raceId", "resultId", "race", "country",
    "nationality_of_circuit", "driverId", "number",
    "fastestLap_ms", "status", "constructor", "positionOrder", "number.1",
    "laps", "major_regulation_change", "time", "milliseconds"
]

# Verify that the columns exist in the DataFrame before dropping them
existing_cols_to_drop = [col for col in cols_to_drop if col in process_df.columns]

# Drop the verified columns from the DataFrame
process_df.drop(columns=existing_cols_to_drop, axis=1, inplace=True)

In [39]:
process_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11235 entries, 0 to 11234
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   race_index                        11235 non-null  int64  
 1   year                              11235 non-null  int64  
 2   driver_name                       11235 non-null  object 
 3   F2_champion                       11235 non-null  int64  
 4   Former_F1_World_Champion          11235 non-null  int64  
 5   Nationality                       11235 non-null  object 
 6   home_race                         11235 non-null  int64  
 7   constructorId                     11235 non-null  int64  
 8   engine_manufacturer               11235 non-null  object 
 9   constructor_nationality           11235 non-null  object 
 10  starting_grid_position            11235 non-null  int64  
 11  points                            11235 non-null  float64
 12  poin

### Changing Data Types

In [40]:
# # Convert columns to the right data types
# df[col] = df[col].astype('string')
# df[col] = df[col].astype('int')
# df[col] = pd.to_datetime(df[col], infer_datetime_format=True)

# # Convert to categorical datatype
# col_cat = ptypes.CategoricalDtype(categories=['A', 'B', 'C'], ordered=True)
# df['col_cat'] = df['col_cat'].astype(col_cat)

In [41]:
# # Verify conversion
# assert ptypes.is_string_dtype(df[col])
# assert ptypes.is_numeric_dtype(df[col])
# cols_to_check = []
# assert all(ptypes.is_datetime64_any_dtype(df[col]) for col in cols_to_check)

### Dropping Duplicates

In [42]:
# # Drop entirely duplicated rows
# df.drop_duplicates(inplace=True, ignore_index=True)

In [43]:
# # Verify rows dropped
# assert df.duplicated().sum()==0

### Handling Unreasonable Data Ranges

In [44]:
# # Drop affected rows
# df = df.loc[~((df['A'] == 0) | (df['B'] > 100))].reset_index()

In [45]:
# # Verify rows dropped
# len(df)

### Feature Engineering / Transformation

#### constructor_points_at_stage_of_season

In [46]:
constuctor_points_sum_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11235 entries, 0 to 11234
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   race_index                        11235 non-null  int64  
 1   year                              11235 non-null  int64  
 2   driver_name                       11235 non-null  object 
 3   F2_champion                       11235 non-null  int64  
 4   Former_F1_World_Champion          11235 non-null  int64  
 5   Nationality                       11235 non-null  object 
 6   home_race                         11235 non-null  int64  
 7   constructorId                     11235 non-null  int64  
 8   engine_manufacturer               11235 non-null  object 
 9   constructor_nationality           11235 non-null  object 
 10  starting_grid_position            11235 non-null  int64  
 11  points                            11235 non-null  float64
 12  poin

In [55]:
constructor_points_sum_df = process_df.copy()

In [56]:
# Correcting the DataFrame name (adding the missing 'r' in 'constructor')
constructor_points_sum_df_2 = constructor_points_sum_df.sort_values(by=['year', 'race_index', 'constructorId'])

# Use a temporary DataFrame to assist with the cumulative sum calculation
temp_df = constructor_points_sum_df_2.groupby(['year', 'constructorId'])['points'].cumsum().reset_index()

# Merge this temporary DataFrame back to the original sorted DataFrame
df_merged = pd.merge(constructor_points_sum_df_2, temp_df, on=['year', 'constructorId'], how='left')

# Rename the column with the cumulative points
df_merged = df_merged.rename(columns={'points_y': 'constructorId_points_at_stage_of_season', 'points_x': 'points'})

# Assuming you want to drop the old corrected points column which was initialized to 0
df_merged.drop(columns=['corrected_constructorId_points'], inplace=True)

# Now df_merged should contain the 'constructorId_points_at_stage_of_season' with the cumulative points

KeyError: 'year'

In [57]:
# First, ensure 'race_index', 'year', and 'constructorId' are sorted in the order we want to process them
constuctor_points_sum_df = constuctor_points_sum_df.sort_values(by=['year', 'race_index', 'constructorId'])

# Initialize a new column for corrected constructorId points
constuctor_points_sum_df['corrected_constructorId_points'] = 0

# Use a temporary DataFrame to assist with the cumulative sum calculation
temp_df = constuctor_points_sum_df.groupby(['year', 'race_index', 'constructorId'])['points'].sum().groupby(level=[0, 2]).cumsum().reset_index()

# Merge this temporary DataFrame back to the original sorted DataFrame
# This step ensures each driver for the constructorId at that race_index sees the summed points on the constructorId level
df_merged = pd.merge(constuctor_points_sum_df, temp_df, on=['year', 'race_index', 'constructorId'], how='left')

# The merged DataFrame now has an additional column with the cumulative points which needs to be renamed and checked
df_merged = df_merged.rename(columns={'points_y': 'constructorId_points_at_stage_of_season', 'points_x': 'points'})

# Drop the previously incorrectly calculated column
df_merged.drop(columns=['corrected_constructorId_points'], inplace=True)

In [66]:
# Check the first few rows to ensure the new column has been correctly calculated
df_merged[['race_index', 'driver_name', 'year', 'constructorId', 'points', 'constructorId_points_at_stage_of_season']].head(50)

,race_index,driver_name,year,constructorId,points,constructorId_points_at_stage_of_season
0,1,Mika Häkkinen,1995,1,3.0,4.0
1,1,Mark Blundell,1995,1,1.0,4.0
2,1,Damon Hill,1995,3,0.0,6.0
3,1,David Coulthard,1995,3,6.0,6.0
4,1,Gerhard Berger,1995,6,4.0,6.0
5,1,Jean Alesi,1995,6,2.0,6.0
6,1,Heinz-Harald Frentzen,1995,15,0.0,0.0
7,1,Karl Wendlinger,1995,15,0.0,0.0
8,1,Eddie Irvine,1995,17,0.0,0.0
9,1,Rubens Barrichello,1995,17,0.0,0.0


In [70]:
# Check the first few rows to ensure the new column has been correctly calculated
df_merged[['race_index', 'driver_name', 'year', 'constructorId', 'points', 'constructorId_points_at_stage_of_season']].tail(50)

,race_index,driver_name,year,constructorId,points,constructorId_points_at_stage_of_season
11185,525,Lance Stroll,2023,117,0.0,168.0
11186,525,Fernando Alonso,2023,117,6.0,168.0
11187,525,George Russell,2023,131,10.0,195.0
11188,525,Lewis Hamilton,2023,131,15.0,195.0
11189,525,Kevin Magnussen,2023,210,0.0,8.0
11190,525,Nico Hülkenberg,2023,210,0.0,8.0
11191,525,Nyck de Vries,2023,213,0.0,2.0
11192,525,Yuki Tsunoda,2023,213,0.0,2.0
11193,525,Pierre Gasly,2023,214,0.0,45.0
11194,525,Esteban Ocon,2023,214,0.0,45.0


That seems to have worked - now we have a column that has the cumulative points for the constructor at this stage of the season for each driver.

#### driver_points_at_stage_of_season

In [71]:
driver_points_sum_df = df_merged.copy()

In [73]:
# Initialize a new column for driver points at stage of season
driver_points_sum_df['driver_points_at_stage_of_season'] = 0

# Use a temporary DataFrame to assist with the cumulative sum calculation for drivers
temp_driver_df = driver_points_sum_df.groupby(['year', 'race_index', 'driver_name'])['points'].sum().groupby(level=[0, 2]).cumsum().reset_index()

# Merge this temporary DataFrame back to the original DataFrame
# This step ensures each driver sees the summed points at that stage of the season
df_merged_with_driver_points = pd.merge(driver_points_sum_df, temp_driver_df, on=['year', 'race_index', 'driver_name'], how='left')

# The merged DataFrame now has an additional column with the cumulative points which needs to be renamed and checked
df_merged_with_driver_points = df_merged_with_driver_points.rename(columns={'points_y': 'driver_points_at_stage_of_season', 'points_x': 'points'})

In [76]:
# Check the first few rows to ensure the new column has been correctly calculated
df_merged_with_driver_points[['race_index', 'year', 'driver_name', 'points', 'driver_points_at_stage_of_season']].head(50)

,race_index,year,driver_name,points,driver_points_at_stage_of_season,driver_points_at_stage_of_season
0,1,1995,Mika Häkkinen,3.0,0,3.0
1,1,1995,Mark Blundell,1.0,0,1.0
2,1,1995,Damon Hill,0.0,0,0.0
3,1,1995,David Coulthard,6.0,0,6.0
4,1,1995,Gerhard Berger,4.0,0,4.0
5,1,1995,Jean Alesi,2.0,0,2.0
6,1,1995,Heinz-Harald Frentzen,0.0,0,0.0
7,1,1995,Karl Wendlinger,0.0,0,0.0
8,1,1995,Eddie Irvine,0.0,0,0.0
9,1,1995,Rubens Barrichello,0.0,0,0.0


In [75]:
# Check the first few rows to ensure the new column has been correctly calculated
df_merged_with_driver_points[['race_index', 'year', 'driver_name', 'points', 'driver_points_at_stage_of_season']].tail(50)

,race_index,year,driver_name,points,driver_points_at_stage_of_season,driver_points_at_stage_of_season
11185,525,2023,Lance Stroll,0.0,0,38.0
11186,525,2023,Fernando Alonso,6.0,0,130.0
11187,525,2023,George Russell,10.0,0,76.0
11188,525,2023,Lewis Hamilton,15.0,0,119.0
11189,525,2023,Kevin Magnussen,0.0,0,2.0
11190,525,2023,Nico Hülkenberg,0.0,0,6.0
11191,525,2023,Nyck de Vries,0.0,0,0.0
11192,525,2023,Yuki Tsunoda,0.0,0,2.0
11193,525,2023,Pierre Gasly,0.0,0,16.0
11194,525,2023,Esteban Ocon,0.0,0,29.0


That seems to have worked - now we have a column that has the cumulative points for the driver at this stage of the season for each driver.

### Other Feature Engineering Ideas

1) team_development_rank_last_year, 
2) statusId_finished_last_race
3) team_rank_first_race_after_major_regulation_change

In [ ]:
# # Get unique values of interested columns
# cols = []
# pd.unique(df[cols].values.ravel('k'))  # argument 'k' lists the values in the order of the cols 

In [ ]:
# # Create custom function
# # Google style docstrings
# # https://sphinxcontrib-napoleon.readthedocs.io/en/latest/example_google.html
# def custom_function(param1: int, param2: str) -> bool:
#     """Example function with PEP 484 type annotations.

#     Args:
#         param1: The first parameter.
#         param2: The second parameter.

#     Returns:
#         The return value. True for success, False otherwise.

#     """

In [ ]:
# # Apply function to multiple columns
# cols = []
# df_updated = df.copy()
# df_updated[cols] = df_updated[cols].applymap(custom_function)

# # Create new aggregated boolean column
# df_updated['bool'] = df_updated[cols].any(axis=1, skipna=False)